In [ ]:
import numpy as np
from typing import List, Tuple

DEFAULT_MORPHOLOGY = np.array(
    [
        # thumb
        [
            # bones lengths
            7.5329e-01,
            3.7025e-01,
            2.5811e-01,
            # start point (x, y, z)
            7.3117e-03,
            1.1166e-02,
            4.2211e-04,
            # starting rotation axis angle (radians)
            -3.5689e00,
        ],
        # index
        [
            # bones lengths
            4.3777e-01,
            2.7459e-01,
            2.2351e-01,
            # starting point (x, y, z)
            -1.6304e-03,
            9.7066e-01,
            2.2563e-01,
            # starting rotation axis angle (radians)
            1.5067e00,
        ],
        # middle
        [
            # bones lengths
            5.3909e-01,
            3.2731e-01,
            2.2931e-01,
            # starting point (x, y, z)
            -9.7050e-04,
            9.9966e-01,
            -1.7299e-02,
            # starting rotation axis angle (radians)
            -1.1042e01,
        ],
        # ring
        [
            # bones lengths
            5.3695e-01,
            3.2312e-01,
            2.3646e-01,
            # starting point (x, y, z)
            -3.7696e-02,
            9.1871e-01,
            -2.2384e-01,
            # starting rotation axis angle (radians)
            1.4743e00,
        ],
        # pinky
        [
            # bones lengths
            4.6647e-01,
            2.6924e-01,
            2.3053e-01,
            # starting point (x, y, z)
            -9.2299e-02,
            7.6455e-01,
            -3.9918e-01,
            # starting rotation axis angle (radians)
            1.3717e00,
        ],
    ],
    dtype=np.float32,
)


def rot(
    v: np.ndarray, p: np.ndarray, alpha: float, beta: float
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Rotate vector `v` and perpendicular vector `p` using angles alpha and beta.
    """
    sinA, cosA = np.sin(alpha), np.cos(alpha)
    sinB, cosB = np.sin(beta), np.cos(beta)

    q = np.cross(v, p)

    v_hat = q * sinA + v * cosA * cosB + p * sinB
    p_hat = p * cosB - v * sinB

    return v_hat, p_hat


def hand_landmarks_by_angles(
    angles: np.ndarray,
    morphology: np.ndarray = DEFAULT_MORPHOLOGY,  # shape (20,)  # shape (5, 7)
) -> List[np.ndarray]:
    """
    Compute hand landmarks from angles and morphology.
    Returns a list of 20 (3,) np.ndarrays
    """
    landmarks = [np.zeros(3, dtype=np.float32) for _ in range(20)]

    angles = angles.reshape(5, 4)

    for i in range(1, 5):
        idx = 4 * i
        landmarks[idx] = morphology[i, 3:6].copy()

    V = np.array([0.0, 1.0, 0.0], dtype=np.float32)
    P = np.array([-1.0, 0.0, 0.0], dtype=np.float32)

    for finger_index in range(5):
        morph = morphology[finger_index]
        local_angles = angles[finger_index]

        base_idx = finger_index * 4
        bone_lengths = morph[0:3]
        joint = morph[3:6].copy()
        gamma = morph[6]

        a0, b0, a2, a3 = local_angles
        chain_angles = [
            (a0, b0),
            (a2, 0.0),
            (a3, 0.0),
        ]

        v = joint / (np.linalg.norm(joint) + 1e-8)

        sinB = np.dot(v, P)
        cosB = np.sqrt(1.0 - sinB**2)
        p = P * cosB - V * sinB

        # Rotate p around v by gamma
        sinG, cosG = np.sin(gamma), np.cos(gamma)
        p = p * cosG + np.cross(p, v) * sinG

        for j in range(3):
            l = bone_lengths[j]
            alpha, beta = chain_angles[j]
            v, p = rot(v, p, alpha, beta)
            joint = joint + v * l
            landmarks[base_idx + j + 1] = joint.copy()

    return landmarks


def irot(
    v: np.ndarray, p: np.ndarray, v_hat: np.ndarray
) -> Tuple[float, float, np.ndarray]:
    """
    Inverse rotation from v, p, v_hat to alpha, beta and p_hat
    """
    q = np.cross(v, p)

    sinA = np.clip(np.dot(v_hat, q), -1 + 1e-6, 1 - 1e-6)
    sinB = np.clip(np.dot(v_hat, p), -1 + 1e-6, 1 - 1e-6)

    alpha = np.arcsin(sinA)
    beta = np.arcsin(sinB)

    cosB = np.sqrt(max(1.0 - sinB**2, 1e-7))
    p_hat = p * cosB - v * sinB

    return alpha, beta, p_hat


def inverse_hand_angles_by_landmarks(
    landmarks: List[np.ndarray],  # list of 20 (3,) arrays
    morphology: np.ndarray = DEFAULT_MORPHOLOGY,  # shape (5, 7)
) -> np.ndarray:
    """
    Returns a (20,) array of angles
    """
    angles = np.zeros((5, 4), dtype=np.float32)

    V = np.array([0.0, 1.0, 0.0], dtype=np.float32)
    P = np.array([-1.0, 0.0, 0.0], dtype=np.float32)

    for i in range(5):
        morph = morphology[i]
        joint = morph[3:6].copy()
        gamma = morph[6]

        v = joint / (np.linalg.norm(joint) + 1e-8)

        sinB = np.dot(v, P)
        cosB = np.sqrt(1.0 - sinB**2)
        p = P * cosB - V * sinB

        sinG, cosG = np.sin(gamma), np.cos(gamma)
        p = p * cosG + np.cross(p, v) * sinG

        idx = [4 * i + j for j in range(4)]

        for j in range(3):
            delta = landmarks[idx[j + 1]] - landmarks[idx[j]]
            target = delta / (np.linalg.norm(delta) + 1e-8)

            alpha, beta, p = irot(v, p, target)
            v = target

            if j == 0:
                angles[i, 0] = alpha
                angles[i, 1] = beta
            else:
                angles[i, j + 1] = alpha

    return angles.flatten()

In [3]:
from typing import List
import numpy as np


def read_hands(file: str):
    poses: List[np.ndarray] = []
    pose_size = 20 * 3  # number of floats
    with open(file, "rb") as f:
        data = f.read()
        total_floats = len(data) // 4  # float32 = 4 bytes
        if total_floats % pose_size != 0:
            raise ValueError("File size is not a multiple of single pose size")
        array = np.frombuffer(data, dtype=np.float32)
        for i in range(0, total_floats, pose_size):
            pose = array[i : i + pose_size].reshape((20, 3))
            poses.append(pose)
    return np.stack(poses, axis=0)

In [ ]:
import plotly.graph_objects as go
import numpy as np
import torch


def plot_3d_hands(landmarks_sequence: torch.Tensor | np.ndarray):
    if isinstance(landmarks_sequence, torch.Tensor):
        landmarks_sequence = landmarks_sequence.detach().cpu().numpy()

    n_frames = landmarks_sequence.shape[0]
    n_pts = landmarks_sequence.shape[1]
    labels = [str(i) for i in range(n_pts)]

    fig = go.Figure()

    # Add traces for the first frame
    fig.add_trace(
        go.Scatter3d(
            x=landmarks_sequence[0, :, 0],
            y=landmarks_sequence[0, :, 1],
            z=landmarks_sequence[0, :, 2],
            mode="markers+text",
            marker=dict(size=4, color="blue"),
            text=labels,
            textposition="top center",
            textfont=dict(size=8, color="black"),
            name="Landmarks",
        )
    )

    connections = [
        (0, 1),
        (1, 2),
        (2, 3),  # Thumb
        (0, 4),
        (4, 5),
        (5, 6),
        (6, 7),  # Index
        (0, 8),
        (8, 9),
        (9, 10),
        (10, 11),  # Middle
        (0, 12),
        (12, 13),
        (13, 14),
        (14, 15),  # Ring
        (0, 16),
        (16, 17),
        (17, 18),
        (18, 19),  # Pinky
        # Palm
        (1, 4),
        (4, 8),
        (8, 12),
        (12, 16),
    ]

    # Add traces for connections for the first frame
    for i, j in connections:
        fig.add_trace(
            go.Scatter3d(
                x=[landmarks_sequence[0, i, 0], landmarks_sequence[0, j, 0]],
                y=[landmarks_sequence[0, i, 1], landmarks_sequence[0, j, 1]],
                z=[landmarks_sequence[0, i, 2], landmarks_sequence[0, j, 2]],
                mode="lines",
                line=dict(color="black", width=2),
                showlegend=False,
            )
        )

    # Create frames for the animation
    frames = [
        go.Frame(
            data=[
                go.Scatter3d(
                    x=landmarks_sequence[k, :, 0],
                    y=landmarks_sequence[k, :, 1],
                    z=landmarks_sequence[k, :, 2],
                    mode="markers+text",
                    marker=dict(size=4, color="blue"),
                    text=labels,
                    textposition="top center",
                    textfont=dict(size=8, color="black"),
                    name="Landmarks",
                )
            ]
            + [
                go.Scatter3d(
                    x=[landmarks_sequence[k, i, 0], landmarks_sequence[k, j, 0]],
                    y=[landmarks_sequence[k, i, 1], landmarks_sequence[k, j, 1]],
                    z=[landmarks_sequence[k, i, 2], landmarks_sequence[k, j, 2]],
                    mode="lines",
                    line=dict(color="black", width=2),
                    showlegend=False,
                )
                for i, j in connections
            ],
            name=str(k),
        )
        for k in range(n_frames)
    ]

    fig.frames = frames

    # Add slider and play/pause button
    sliders = [
        {
            "pad": {"b": 10, "t": 50},
            "len": 0.9,
            "x": 0.1,
            "xanchor": "left",
            "y": 0,
            "yanchor": "top",
            "steps": [
                {
                    "args": [
                        [f.name],
                        {
                            "frame": {"duration": 33.33, "redraw": True},
                            "mode": "immediate",
                            "transition": {"duration": 0},
                        },
                    ],
                    "label": str(k),
                    "method": "animate",
                }
                for k, f in enumerate(frames)
            ],
        }
    ]

    fig.update_layout(
        updatemenus=[
            {
                "buttons": [
                    {
                        "args": [
                            None,
                            {
                                "frame": {"duration": 33.33, "redraw": True},
                                "fromcurrent": True,
                                "transition": {"duration": 0},
                            },
                        ],
                        "label": "Play",
                        "method": "animate",
                    },
                    {
                        "args": [
                            [None],
                            {
                                "frame": {"duration": 0, "redraw": True},
                                "mode": "immediate",
                                "transition": {"duration": 0},
                            },
                        ],
                        "label": "Pause",
                        "method": "animate",
                    },
                ],
                "direction": "left",
                "pad": {"r": 10, "t": 87},
                "showactive": False,
                "type": "buttons",
                "x": 0.1,
                "xanchor": "right",
                "y": 0,
                "yanchor": "top",
            }
        ],
        sliders=sliders,
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            xaxis=dict(autorange="reversed"),  # This line inverts the X axis
        ),
        title="3D Hand Pose with Point Labels",
        margin=dict(l=0, r=0, b=0, t=30),
    )

    fig.show()

In [ ]:
import numpy as np
from scipy.interpolate import interp1d
import numpy as np
from scipy.interpolate import make_interp_spline
from scipy.interpolate import CubicSpline


def _resample_linear(array: np.ndarray, target_size: int) -> np.ndarray:
    """
    Resample a signal array to a target size using linear interpolation

    Args:
        array (np.ndarray): Input signal array of shape (n_samples, ...)
        target_size (int): Desired number of samples in output

    Returns:
        np.ndarray: Resampled array of shape (target_size, ...)
    """
    # Generate x coordinates for original and target
    x = np.linspace(0, 1, array.shape[0])
    x_new = np.linspace(0, 1, target_size)

    # Preserve shape of additional dimensions
    orig_shape = array.shape
    n_channels = np.prod(orig_shape[1:]) if len(orig_shape) > 1 else 1

    # Reshape to 2D array (samples, channels)
    y = array.reshape(-1, n_channels)

    # Create interpolation function
    f = interp1d(x, y, axis=0, kind="linear")

    # Interpolate
    resampled = f(x_new)

    # Reshape back to original dimensions
    if len(orig_shape) > 1:
        resampled = resampled.reshape((target_size,) + orig_shape[1:])

    return resampled


def _resample_bspline(
    array: np.ndarray, target_size: int, order: int = 3
) -> np.ndarray:
    """
    Resample a signal array to a target size using B-spline interpolation

    Args:
        array (np.ndarray): Input signal array of shape (n_samples, ...)
        target_size (int): Desired number of samples in output
        order (int): Order of the B-spline

    Returns:
        np.ndarray: Resampled array of shape (target_size, ...)
    """
    # Generate x coordinates for original and target
    x = np.linspace(0, 1, array.shape[0])
    x_new = np.linspace(0, 1, target_size)

    # Preserve shape of additional dimensions
    orig_shape = array.shape
    n_channels = np.prod(orig_shape[1:]) if len(orig_shape) > 1 else 1

    # Reshape to 2D array (samples, channels)
    y = array.reshape(-1, n_channels)

    # Initialize array for resampled data
    resampled = np.zeros((target_size, n_channels))

    # Fit B-spline for each channel separately
    for ch in range(n_channels):
        spl = make_interp_spline(x, y[:, ch], k=order)
        resampled[:, ch] = spl(x_new)

    # Reshape back to original dimensions
    if len(orig_shape) > 1:
        resampled = resampled.reshape((target_size,) + orig_shape[1:])

    return resampled


def _resample_cubic_spline(array: np.ndarray, target_size: int) -> np.ndarray:
    """
    Resample a signal array to a target size using cubic spline interpolation

    Args:
        array (np.ndarray): Input signal array of shape (n_samples, ...)
        target_size (int): Desired number of samples in output

    Returns:
        np.ndarray: Resampled array of shape (target_size, ...)
    """
    # Generate x coordinates for original and target
    x = np.linspace(0, 1, array.shape[0])
    x_new = np.linspace(0, 1, target_size)

    # Preserve shape of additional dimensions
    orig_shape = array.shape
    n_channels = np.prod(orig_shape[1:]) if len(orig_shape) > 1 else 1

    # Reshape to 2D array (samples, channels)
    y = array.reshape(-1, n_channels)

    # Create cubic spline interpolation function
    cs = CubicSpline(x, y, axis=0)

    # Interpolate
    resampled = cs(x_new)

    # Reshape back to original dimensions
    if len(orig_shape) > 1:
        resampled = resampled.reshape((target_size,) + orig_shape[1:])

    return resampled

In [ ]:
rec = read_hands("../dataset.rec")

In [89]:
import time


def test_resample(start, stop, resample, k, plot=False):
    joint_samples = rec[start:stop]
    angle_samples = np.stack(
        [inverse_hand_angles_by_landmarks(l) for l in joint_samples]
    )

    start_time = time.time()
    angle_samples = resample(angle_samples, angle_samples.shape[0] // k)
    angle_samples = resample(angle_samples, angle_samples.shape[0] * k)
    end_time = time.time()

    joint_samples = np.stack([hand_landmarks_by_angles(a) for a in angle_samples])
    if plot:
        plot_3d_hands(joint_samples)
    else:
        print(f"{resample} resampling time: {end_time - start_time:.4f} seconds")

In [91]:
test_resample(0, 1000, _resample_linear, 4)
test_resample(0, 1000, _resample_bspline, 4)
test_resample(0, 1000, _resample_cubic_spline, 4)

test_resample(200, 300, _resample_linear, 10, plot=True)
test_resample(200, 300, _resample_bspline, 10, plot=True)
test_resample(200, 300, _resample_cubic_spline, 10, plot=True)

<function _resample_linear at 0x0000015569502B00> resampling time: 0.0014 seconds
<function _resample_bspline at 0x00000155695024D0> resampling time: 0.0078 seconds
<function _resample_cubic_spline at 0x0000015569502CB0> resampling time: 0.0020 seconds
